In [1]:
import os
import requests
import gradio as gr

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --------------------------------------------------
# 1️⃣ DRUG NAME DETECTION (FIXED)
# --------------------------------------------------

KNOWN_DRUGS = [
    "acetaminophen", "paracetamol", "ibuprofen",
    "metformin", "amoxicillin", "aspirin"
]

DRUG_SYNONYMS = {
    "paracetamol": "acetaminophen",
    "crocin": "acetaminophen",
    "calpol": "acetaminophen",
    "tylenol": "acetaminophen",
    "brufen": "ibuprofen"
}

def extract_drug_name(query: str):
    query = query.lower()
    for drug in KNOWN_DRUGS:
        if drug in query:
            return DRUG_SYNONYMS.get(drug, drug)
    return None

# --------------------------------------------------
# 2️⃣ OPENFDA MASTER DRUG DATABASE
# --------------------------------------------------

def fetch_drug_from_openfda(drug_name):
    url = (
        "https://api.fda.gov/drug/label.json?"
        f"search=openfda.generic_name:{drug_name}&limit=1"
    )

    response = requests.get(url)
    if response.status_code != 200:
        return None

    data = response.json()["results"][0]

    return {
        "drug": drug_name,
        "indications": " ".join(data.get("indications_and_usage", [])),
        "dosage": " ".join(data.get("dosage_and_administration", [])),
        "warnings": " ".join(data.get("warnings", [])),
        "interactions": " ".join(data.get("drug_interactions", []))
    }

# --------------------------------------------------
# 3️⃣ DOCUMENT RAG (OPTIONAL CONTEXT)
# --------------------------------------------------

PDF_URL = "https://dailymed.nlm.nih.gov/dailymed/getFile.cfm?setid=49a0b5c2-ebaf-4c4c-905f-dfd1962ac647&type=pdf"
PDF_FILE = "clinical_docs.pdf"

def prepare_vectorstore():
    if not os.path.exists(PDF_FILE):
        r = requests.get(PDF_URL)
        with open(PDF_FILE, "wb") as f:
            f.write(r.content)

    loader = PyPDFLoader(PDF_FILE)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
    chunks = splitter.split_documents(docs)

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    return FAISS.from_documents(chunks, embeddings)

vectorstore = prepare_vectorstore()
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# --------------------------------------------------
# 4️⃣ LLM
# --------------------------------------------------

llm = ChatOllama(model="llama3.2:1b", temperature=0)

# --------------------------------------------------
# 5️⃣ MAIN CHATBOT FUNCTION (FIXED)
# --------------------------------------------------

def pharmacist_bot(query, history):
    drug_name = extract_drug_name(query)

    if not drug_name:
        return "❌ Could not identify a drug name in your question."

    drug_data = fetch_drug_from_openfda(drug_name)

    if not drug_data:
        return f"❌ No verified FDA drug data found for {drug_name}."

    docs = retriever.invoke(query)
    context = "\n\n".join(doc.page_content for doc in docs)

    prompt = ChatPromptTemplate.from_template("""
You are a Pharmacist & Clinical Research Assistant.

Rules:
- Use ONLY verified drug data provided
- Do NOT hallucinate
- If data is missing, clearly say so

DRUG DATA:
Drug: {drug}
Indications: {indications}
Dosage: {dosage}
Warnings: {warnings}
Interactions: {interactions}

DOCUMENT CONTEXT:
{context}

Question: {question}

Answer in a clear, pharmacist-friendly way.
""")

    chain = prompt | llm | StrOutputParser()

    return chain.invoke({
        "drug": drug_data["drug"],
        "indications": drug_data["indications"],
        "dosage": drug_data["dosage"],
        "warnings": drug_data["warnings"],
        "interactions": drug_data["interactions"],
        "context": context,
        "question": query
    })

# --------------------------------------------------
# 6️⃣ GRADIO INTERFACE
# --------------------------------------------------

demo = gr.ChatInterface(
    fn=pharmacist_bot,
    title="AI Pharmacist Assistant",
    description="FDA-verified drug information with clinical document support",
    examples=[
        "What is the maximum daily dose of acetaminophen?",
        "Paracetamol dosage for adults",
        "Metformin contraindications",
        "Ibuprofen drug interactions"
    ]
)

if __name__ == "__main__":
    demo.launch(share=True)


C:\Users\SHAIK SAAJITH\PycharmProjects\JupyterProject1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\SHAIK SAAJITH\PycharmProjects\JupyterProject1\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://56a8b5a5cbd5eeed80.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
